In [2]:
import pandas as pd;
import datetime as dt;
import numpy as np;
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import scale
import pickle

In [3]:
weather = pd.read_csv("weather (table 7)_training_update.csv")
trajectories = pd.read_csv("trajectories(table 5)_training.csv")

#trajectories

del trajectories['vehicle_id']
del trajectories['travel_seq']

trajectories['starting_time'] =  pd.to_datetime(trajectories['starting_time'], format = "%Y-%m-%d %H:%M:%S")

st = []

for c in trajectories['starting_time']:
    st.append(c - dt.timedelta(seconds = c.second, minutes = c.minute))
    
trajectories['starting_time'] = st

trajectories = trajectories.groupby(['intersection_id', 'tollgate_id', 'starting_time']).mean()

trajectories = trajectories.reset_index()

datehour = []

for c in trajectories['starting_time']:
    datehour.append(c - dt.timedelta(hours = c.hour % 3, minutes = c.minute, seconds = c.second))
    
trajectories['datehour'] = datehour

#weahter

weather['datehour'] = pd.to_timedelta(weather['hour'], unit="H") + pd.to_datetime(weather['date'], format="%Y-%m-%d")

del weather['hour']
del weather['date']

#merge
training = pd.merge(trajectories, weather, on = 'datehour')

del training['datehour']

training = training.sort_values(by = ['intersection_id', 'tollgate_id', 'starting_time'])

M = []
H = []
w = []

for c in training['starting_time']:
    w.append(int(dt.date.strftime(c, format="%w")));
    H.append(c.hour)
    M.append(c.minute)
    
training['hour'] = H
training['week'] = w
training['minute'] = M

training.loc[training['week']==0, 'week'] = 7
training.loc[training['intersection_id']=='A', 'intersection_id'] = 1
training.loc[training['intersection_id']=='B', 'intersection_id'] = 2
training.loc[training['intersection_id']=='C', 'intersection_id'] = 3

del training['starting_time']

cols = training.columns

tmp = cols[:cols.get_loc('travel_time')];
tmp = tmp.append(cols[cols.get_loc('travel_time') + 1:])
tmp = tmp.append(cols[cols.get_loc('travel_time'):(cols.get_loc('travel_time')+1)])

training.dropna()

training = training[tmp]

training.to_csv("training.csv")


In [4]:
import re

submission = pd.read_csv("submission_sample_travelTime.csv")
testing_weather = pd.read_csv("weather (table 7)_test1.csv")

#submission
datetime = []

for c in submission['time_window']:
    c = c.translate(None, '[)')
    datetime.append(c.split(',')[0])
    
submission['datetime'] = datetime;
submission['datetime'] =  pd.to_datetime(submission['datetime'], format = "%Y-%m-%d %H:%M:%S")

datehour = []

for c in submission['datetime']:
    datehour.append(c - dt.timedelta(hours = c.hour % 3, minutes = c.minute))
    
submission['datehour'] = datehour

#weahter

testing_weather['datehour'] = pd.to_timedelta(testing_weather['hour'], unit="H") + pd.to_datetime(testing_weather['date'], format="%Y-%m-%d")

del testing_weather['hour']
del testing_weather['date']

#merge
submission = pd.merge(submission, testing_weather, on = 'datehour')

submission.to_csv('submission_raw.csv')

del submission['datehour']

M = []
H = []
w = []

for c in submission['datetime']:
    w.append(int(dt.date.strftime(c, format="%w")));
    H.append(c.hour)
    M.append(c.minute)
    
submission['hour'] = H
submission['week'] = w
submission['minute'] = M


submission.loc[submission['week']==0, 'week'] = 7
submission.loc[submission['intersection_id']=='A', 'intersection_id'] = 1
submission.loc[submission['intersection_id']=='B', 'intersection_id'] = 2
submission.loc[submission['intersection_id']=='C', 'intersection_id'] = 3

del submission['datetime']

cols = submission.columns

tmp = cols[:cols.get_loc('avg_travel_time')];
tmp = tmp.append(cols[cols.get_loc('avg_travel_time') + 1:])
tmp = tmp.append(cols[cols.get_loc('avg_travel_time'):(cols.get_loc('avg_travel_time')+1)])

submission.dropna()

submission = submission[tmp]

del submission['time_window']

submission.to_csv("submission.csv")


In [5]:
weather = pd.read_csv("weather (table 7)_test1.csv")
trajectories = pd.read_csv("trajectories(table 5)_test1.csv")

#trajectories

del trajectories['vehicle_id']
del trajectories['travel_seq']

trajectories['starting_time'] =  pd.to_datetime(trajectories['starting_time'], format = "%Y-%m-%d %H:%M:%S")

st = []

for c in trajectories['starting_time']:
    st.append(c - dt.timedelta(seconds = c.second, minutes = c.minute))
    
trajectories['starting_time'] = st

trajectories = trajectories.groupby(['intersection_id', 'tollgate_id', 'starting_time']).mean()

trajectories = trajectories.reset_index()

datehour = []

for c in trajectories['starting_time']:
    datehour.append(c - dt.timedelta(hours = c.hour % 3, minutes = c.minute, seconds = c.second))
    
trajectories['datehour'] = datehour

#weahter

weather['datehour'] = pd.to_timedelta(weather['hour'], unit="H") + pd.to_datetime(weather['date'], format="%Y-%m-%d")

del weather['hour']
del weather['date']

#merge
testing = pd.merge(trajectories, weather, on = 'datehour')

del testing['datehour']

testing = testing.sort_values(by = ['intersection_id', 'tollgate_id', 'starting_time'])

M = []
H = []
w = []

for c in testing['starting_time']:
    w.append(int(dt.date.strftime(c, format="%w")));
    H.append(c.hour)
    M.append(c.minute)
    
testing['hour'] = H
testing['week'] = w
testing['minute'] = M

testing.loc[testing['week']==0, 'week'] = 7
testing.loc[testing['intersection_id']=='A', 'intersection_id'] = 1
testing.loc[testing['intersection_id']=='B', 'intersection_id'] = 2
testing.loc[testing['intersection_id']=='C', 'intersection_id'] = 3

del testing['starting_time']

cols = testing.columns

tmp = cols[:cols.get_loc('travel_time')];
tmp = tmp.append(cols[cols.get_loc('travel_time') + 1:])
tmp = tmp.append(cols[cols.get_loc('travel_time'):(cols.get_loc('travel_time')+1)])

testing.dropna()

testing = testing[tmp]

testing.to_csv("testing.csv")